## Часть 1

###### 1.Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [1]:
SELECT COUNT(DISTINCT title)
FROM stackoverflow.posts 
WHERE score > 300 OR favorites_count >= 100

SyntaxError: invalid syntax (2853178450.py, line 1)

###### 2.Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
WITH df AS(
    SELECT COUNT(p.id) AS cnt,
    CAST(DATE_TRUNC('day',creation_date) AS date)
    FROM stackoverflow.posts AS p
    INNER JOIN stackoverflow.post_types AS pt ON p.post_type_id=pt.id
    WHERE CAST(p.creation_date AS date) BETWEEN '2008-11-01' AND '2008-11-18'
    AND pt.type = 'Question'
    GROUP BY CAST(DATE_TRUNC('day',creation_date) AS date)
    ORDER BY CAST(DATE_TRUNC('day',creation_date) AS date)
)

SELECT
ROUND(AVG(cnt),0)
FROM df

###### 3.Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.badges AS b
JOIN stackoverflow.users AS u ON b.user_id=u.id
WHERE b.creation_date::date = u.creation_date::date;

###### 4.Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.posts AS p 
JOIN stackoverflow.votes AS v ON p.id=v.post_id
JOIN stackoverflow.users AS u ON p.user_id = u.id
WHERE u.display_name = 'Joel Coehoorn' 

###### 5.Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
RANK() OVER (ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id

###### 6.Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT u.id,
COUNT(u.id) AS cnt
FROM stackoverflow.users AS u
JOIN stackoverflow.votes AS v ON u.id = v.user_id
JOIN stackoverflow.vote_types AS vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY u.id
ORDER BY cnt DESC, u.id DESC
LIMIT 10

###### 7.Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.Отобразите несколько полей:идентификатор пользователя;число значков;место в рейтинге — чем больше значков, тем выше рейтинг.Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
WITH df AS (
SELECT user_id,
    COUNT(id) AS cnt
    FROM stackoverflow.badges
    WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15' 
    GROUP BY user_id
)

SELECT user_id,
cnt,
DENSE_RANK () OVER (ORDER BY cnt DESC)
FROM df
ORDER BY cnt DESC,user_id
LIMIT 10

###### 8.Сколько в среднем очков получает пост каждого пользователя?Сформируйте таблицу из следующих полей:заголовок поста;идентификатор пользователя;число очков поста;среднее число очков пользователя за пост, округлённое до целого числа.Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
WITH df AS(
    SELECT *
FROM stackoverflow.posts
WHERE title IS NOT NULL AND score != 0
    )
    
SELECT title,
user_id,
score,
ROUND(AVG(score) OVER (PARTITION BY user_id)) AS avg
FROM df

###### 9.Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT title
FROM stackoverflow.posts
WHERE user_id IN (
                  SELECT user_id
                  FROM stackoverflow.badges
                  GROUP BY user_id
                  HAVING COUNT(id) >1000
                  )
  AND title IS NOT NULL;

###### 10.Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей: пользователям с числом просмотров больше либо равным 350 присвойте группу 1; пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2; пользователям с числом просмотров меньше 100 — группу 3. Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.

In [ ]:
SELECT id,
views,
CASE 
    WHEN views >= 350 THEN 1
    WHEN views < 350 AND views >= 100 THEN 2
    WHEN views < 100 THEN 3
    END
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views != 0

###### 11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH df1 AS(
    SELECT id,
views,
CASE 
    WHEN views >= 350 THEN 1
    WHEN views < 350 AND views >= 100 THEN 2
    WHEN views < 100 THEN 3
    END category
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views != 0
ORDER BY views DESC,id
    ),
    
df2 AS (SELECT id,
views,
category,
MAX(views) OVER (PARTITION BY category) as max
FROM df1)

SELECT id,
category,
views
FROM df2
WHERE views = max
ORDER BY views DESC,id


###### 12.Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями: номер дня; число пользователей, зарегистрированных в этот день; сумму пользователей с накоплением.

In [ ]:
WITH df AS (SELECT COUNT(id) AS cnt_id,
EXTRACT(DAY FROM creation_date) As day
FROM stackoverflow.users
WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
GROUP BY EXTRACT(DAY FROM creation_date)
           )
           
SELECT day,
cnt_id,
SUM(cnt_id) OVER(ORDER BY day)
FROM df


###### 13.Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите: идентификатор пользователя; разницу во времени между регистрацией и первым постом.

In [ ]:
WITH df AS (SELECT DISTINCT user_id ,
MIN(creation_date) OVER (PARTITION BY user_id) AS diff
FROM stackoverflow.posts
)


SELECT df.user_id,
df.diff - u.creation_date AS raz
FROM stackoverflow.users AS u
JOIN df ON u.id=df.user_id

## Часть 2

###### 1.Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT 
SUM(views_count),
CAST(DATE_TRUNC('month',creation_date) AS date)
FROM stackoverflow.posts
WHERE CAST(DATE_TRUNC('month',creation_date) AS date) BETWEEN '2008-01-01' AND '2008-12-01'
GROUP BY CAST(DATE_TRUNC('month',creation_date) AS date)
ORDER BY SUM(views_count) DESC

###### 2.Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
COUNT(DISTINCT p.user_id)
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id=p.user_id
JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
WHERE p.creation_date::date BETWEEN u.creation_date::date AND (u.creation_date::date + INTERVAL '1 month')
   AND pt.type LIKE '%Answer%'
GROUP BY u.display_name
HAVING COUNT(p.id) > 100
ORDER BY u.display_name

###### 3.Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
SELECT DATE_TRUNC('month', creation_date)::date,
       COUNT(DISTINCT id)
FROM
(SELECT p.creation_date,
        p.id
FROM stackoverflow.posts p
JOIN (SELECT u.id
     FROM stackoverflow.users u
     JOIN stackoverflow.posts p
     ON u.id=p.user_id
     WHERE p.creation_date::date BETWEEN '2008-12-01' AND '2008-12-31'
     AND u.creation_date::date BETWEEN '2008-09-01' AND '2008-09-30') u
ON p.user_id=u.id) t1
WHERE EXTRACT(YEAR FROM creation_date) = 2008
GROUP BY 1
ORDER BY 1 DESC

###### 4.Используя данные о постах, выведите несколько полей:идентификатор пользователя, который написал пост;дата создания поста;количество просмотров у текущего поста;сумма просмотров постов автора с накоплением.Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [2]:
SELECT user_id,
creation_date,
views_count,
SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY user_id,creation_date

SyntaxError: invalid syntax (1856271491.py, line 1)

###### 5.Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
SELECT ROUND(AVG(df.cnt))
FROM (
      SELECT user_id,
             COUNT(DISTINCT creation_date::date)  AS cnt
      FROM stackoverflow.posts
      WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07' 
      GROUP BY user_id
) AS df

###### 6.На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:Номер месяца.Количество постов за месяц.Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

In [ ]:
WITH df AS(
    SELECT COUNT(DISTINCT id) AS cnt,
DATE_TRUNC('month',creation_date) :: date AS dt
FROM stackoverflow.posts
WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
GROUP BY DATE_TRUNC('month',creation_date) :: date
)


SELECT EXTRACT(MONTH FROM dt) AS number_dt,
cnt,
ROUND((cnt ::numeric / LAG(cnt) OVER (ORDER BY EXTRACT(MONTH FROM dt)) - 1) *100,2) AS percent
FROM df

###### 7.Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:номер недели;дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
WITH cnt_posts AS (SELECT user_id,
       COUNT(DISTINCT id) AS cnt
FROM stackoverflow.posts
GROUP BY user_id
ORDER BY cnt DESC
LIMIT 1),

dt AS (
SELECT p.user_id,
p.creation_date,
EXTRACT('week' FROM p.creation_date) AS week_number
FROM stackoverflow.posts AS p
JOIN cnt_posts ON cnt_posts.user_id = p.user_id
WHERE DATE_TRUNC('month', p.creation_date)::date = '2008-10-01'
)

SELECT DISTINCT week_number :: numeric,
MAX(creation_date) OVER (ORDER BY week_number)
FROM dt